### 【 윤동주 시인의 시 제목과 내용 추출 및 저장 】
- csv 파일로 저장

In [11]:
from urllib.request import urlopen, Request
from urllib.parse import urljoin
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
# 1) 페이지 가져오기
URL = "https://ko.wikisource.org/wiki/%EC%A0%80%EC%9E%90:%EC%9C%A4%EB%8F%99%EC%A3%BC"
HEADERS = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'}

req = Request(URL, headers=HEADERS)
html = urlopen(req)
soup = BeautifulSoup(html, 'html.parser')

In [13]:
# 2) a 태그 전체에서 /wiki/ 로 시작하는 링크만 뽑기
poem_links = []

for a in soup.find_all('a'):
    href = a.get('href')
    title = a.get_text(strip=True)

    # 기본적인 필터만 적용
    if href and href.startswith('/wiki/') and ':' not in href:
        full_url = urljoin(URL, href)
        poem_links.append((title, full_url))

print("시 링크 개수:", len(poem_links))
print(poem_links[:5])   # 일부 확인

시 링크 개수: 96
[('하늘과 바람과 별과 시', 'https://ko.wikisource.org/wiki/%ED%95%98%EB%8A%98%EA%B3%BC_%EB%B0%94%EB%9E%8C%EA%B3%BC_%EB%B3%84%EA%B3%BC_%EC%8B%9C_(1948%EB%85%84)'), ('하늘과 바람과 별과 시', 'https://ko.wikisource.org/wiki/%ED%95%98%EB%8A%98%EA%B3%BC_%EB%B0%94%EB%9E%8C%EA%B3%BC_%EB%B3%84%EA%B3%BC_%EC%8B%9C_(1955%EB%85%84)'), ('하늘과 바람과 별과 시', 'https://ko.wikisource.org/wiki/%ED%95%98%EB%8A%98%EA%B3%BC_%EB%B0%94%EB%9E%8C%EA%B3%BC_%EB%B3%84%EA%B3%BC_%EC%8B%9C_(1979%EB%85%84)'), ('서시', 'https://ko.wikisource.org/wiki/%ED%95%98%EB%8A%98%EA%B3%BC_%EB%B0%94%EB%9E%8C%EA%B3%BC_%EB%B3%84%EA%B3%BC_%EC%8B%9C_(1948%EB%85%84)/%EC%84%9C%EC%8B%9C'), ('자화상', 'https://ko.wikisource.org/wiki/%ED%95%98%EB%8A%98%EA%B3%BC_%EB%B0%94%EB%9E%8C%EA%B3%BC_%EB%B3%84%EA%B3%BC_%EC%8B%9C_(1948%EB%85%84)/%EC%9E%90%ED%99%94%EC%83%81')]


In [16]:
# 3) 상세 페이지 들어가서 시 내용 추출
data = []

for title, link in poem_links:
    req2 = Request(link, headers=HEADERS)
    html2 = urlopen(req2)
    soup2 = BeautifulSoup(html2, 'html.parser')

    body = soup2.select_one("#mw-content-text > div.mw-parser-output")
    if body:
        content = body.get_text("\n", strip=True)
    else:
        content = ""

    data.append([title, content, link])

In [ ]:
# data 값 확인
print(f'data 개수 : {len(data)}')
print(data[0])

data 개수 : 96


['하늘과 바람과 별과 시',
 '이 문헌의 다른 판본을 보시려면\n하늘과 바람과 별과 시\n문서를 참고하십시오.\n←\n하늘과 바람과 별과 시(하늘과바람과별과詩)\n(\n1948\n)\n저자:\n윤동주\n→\n자매 프로젝트\n:\n위키백과 문서\n,\n위키데이터 항목\n윤동주의 사후에 출판된 유고시집. 1941년에 19편이 완성되어 시집으로 펴내려던 것을 일제의 검열을 우려하여 이루지 못하고 1948년 정음사(正音社)에서 유작 30편을 모아 동일한 이름으로 간행하였다. 처음 19편의 시는 모두 연희전문학교 시절에 쓰였다.\n17209\n윤동주\n\u200b\n차 례\n序詩\n하늘과 바람과 별과 詩\n自畵像\n············································································································································································································································································································\n16\n少年\n············································································································································································································································································································\n18\n눈 오는 地圖\n······························

In [19]:
# 4) CSV 저장
df = pd.DataFrame(data, columns=["제목", "내용", "URL"])
df.to_csv("윤동주_시모음.csv", index=False, encoding="utf-8-sig")

print("CSV 저장 완료!")
df.head()

CSV 저장 완료!


,제목,내용,URL
0,하늘과 바람과 별과 시,이 문헌의 다른 판본을 보시려면\n하늘과 바람과 별과 시\n문서를 참고하십시오.\n...,https://ko.wikisource.org/wiki/%ED%95%98%EB%8A...
1,하늘과 바람과 별과 시,이 문헌의 다른 판본을 보시려면\n하늘과 바람과 별과 시\n문서를 참고하십시오.\n...,https://ko.wikisource.org/wiki/%ED%95%98%EB%8A...
2,하늘과 바람과 별과 시,이 문헌의 다른 판본을 보시려면\n하늘과 바람과 별과 시\n문서를 참고하십시오.\n...,https://ko.wikisource.org/wiki/%ED%95%98%EB%8A...
3,서시,←\n하늘과 바람과 별과 시\n(\n1941\n)\n서시(序詩)\n저자:\n윤동주\...,https://ko.wikisource.org/wiki/%ED%95%98%EB%8A...
4,자화상,←\n서시\n하늘과 바람과 별과 시\n(\n1939\n)\n자화상(自畵像)\n저자:...,https://ko.wikisource.org/wiki/%ED%95%98%EB%8A...
